# Sentiment Analysis on Movie Reviews

https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews

영화에 대한 **리뷰와 평점**을 가지고 학습 모델을 만들어 새로운 리뷰가 들어왔을 때 **평점**을 알아내기 위한 예제

## 사용되는 파일
1) Input Data
  * input/train.tsv : 머신러닝 모델 학습을 위한 데이터
  * input/test.tsv : 머신러닝 모델 예측 및 검증을 위한 데이터
  * input/sampleSubmission.tsv : Kaggle 정답 제출용 예제 파일

2) Output Data
  * result.csv : 결과 파일
  * vocabulary.csv : 머신러닝 모델에서 사용된 단어 
  * baseline-submit.csv : Kaggle 정답 제출용 파일


In [2]:
import pandas as pd

## Load Dataset
데이터를 로드하는 과정 

In [3]:
#[arguments]
#first : 파일 경로
#sep : 구분자 (여기서는 Tab)
#index_col : 기본 Index Column
train = pd.read_csv("./input/train.tsv", sep="\t", index_col="PhraseId")

#데이터의 모양 표
print(train.shape) 
#데이터의 상위 5개 행 
train.head() 

(156060, 3)


,SentenceId,Phrase,Sentiment
PhraseId,,,
1,1,A series of escapades demonstrating the adage ...,1
2,1,A series of escapades demonstrating the adage ...,2
3,1,A series,2
4,1,A,2
5,1,series,2


In [4]:
#[arguments]
#first : 파일 경로
#sep : 구분자 (여기서는 Tab)
#index_col : 기본 Index Column
test = pd.read_csv("./input/test.tsv", sep="\t", index_col="PhraseId")

#데이터의 모양 표시
print(test.shape) 
#데이터의 상위 5개 행 
test.head() 

(66292, 2)


,SentenceId,Phrase
PhraseId,,
156061,8545,An intermittently pleasing but mostly routine ...
156062,8545,An intermittently pleasing but mostly routine ...
156063,8545,An
156064,8545,intermittently pleasing but mostly routine effort
156065,8545,intermittently pleasing but mostly routine


## Preprocessing
모델학습 전에 데이터를 가공하기 위한 단계

In [5]:
#clean_text 함수를 정의한다. 문자열을 동일한 방식으로 변경하는 방법
#점수를 가장 높이기 쉬운 방법이지만 머신러닝 학습 용도로 좋은 방법은 아님. 
def clean_text(phrase): 
    # Can not과 Can't 두 가지 표현을 한가지로 통일하기 위한 작업
    phrase = phrase.replace("ca n't", "can not")
    phrase = phrase.replace("does n't", "does not") 
    phrase = phrase.replace("n't", "not")
    phrase = phrase.replace("-", " ")
    phrase = phrase.replace("ed ", " ")
    phrase = phrase.replace("ing ", " ")
    phrase = phrase.replace("s ", " ")
    
    return phrase

In [6]:
#clean_text 함수를 이용하여 기존의 문자열을 정제
train["Phrase(Origin)"] = train["Phrase"].copy() #원본 데이터
train["Phrase"] = train["Phrase"].apply(clean_text) #바뀐 데이터

print(train.shape)
train[["Phrase", "Phrase(Origin)"]].head()

(156060, 4)


,Phrase,Phrase(Origin)
PhraseId,,
1,A serie of escapade demonstrat the adage that ...,A series of escapades demonstrating the adage ...
2,A serie of escapade demonstrat the adage that ...,A series of escapades demonstrating the adage ...
3,A series,A series
4,A,A
5,series,series


In [7]:
#clean_text 함수를 이용하여 기존의 문자열을 정제
test["Phrase(Origin)"] = test["Phrase"].copy() #원본 데이터
test["Phrase"] = test["Phrase"].apply(clean_text) #바뀐 데이터

print(test.shape)
test[["Phrase", "Phrase(Origin)"]].head()

(66292, 3)


,Phrase,Phrase(Origin)
PhraseId,,
156061,An intermittently plea but mostly routine effo...,An intermittently pleasing but mostly routine ...
156062,An intermittently plea but mostly routine effort,An intermittently pleasing but mostly routine ...
156063,An,An
156064,intermittently plea but mostly routine effort,intermittently pleasing but mostly routine effort
156065,intermittently plea but mostly routine,intermittently pleasing but mostly routine


## Vectorize Phrases
자연어 처리를 기능을 제공. 문자열 또는 이미지 데이터를 필요한 데이터로 변환 

#### CountVectorizer Parameters
http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

* max_features : 많이 사용되는 단어 X개만 사용한다 (여기서는 10000으로 사용하였음)
* ngram_range : Value를 구분하는 단위를 설정한다. 
* stop_words : 명시한 단어에 대한 Vectorize를 하지 않음.(This, a, an 등 불필요하다고 판단되는 데이터) 
* lowercase : 문자열을 토큰화(tokenizing)하기 전에 모든 문자열을 소문자로 변환
* token_pattern : 토큰을 구분하기 위한 기준. 정규표현식으로 되어있음 (숫자 제외하고 뽑으려면 -> '(?u)\\b[A-Za-z_][A-Za-z_]+\\b')
* max_df : 모르겠음
* min_df : 모르겠음
* vocabulary : Input 데이터를 지정한다
* binary : 사용되는 단어의 Count 중요도를 측정하지 않음

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=10000, ngram_range=(1, 2), token_pattern='(?u)\\b[A-Za-z_][A-Za-z_]+\\b') #빈도가 낮은 것들은 걸러준다. 최종적으로 상위 1000개의 데이터를 남긴다.

In [10]:
# 
vectorizer.fit(train["Phrase"])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b[A-Za-z_][A-Za-z_]+\\b',
        tokenizer=None, vocabulary=None)

In [11]:
#
X_train = vectorizer.transform(train["Phrase"])

In [12]:
# 컬럼 Feature를 표기하기 위해 데이터를 가져옴
vocabulary = vectorizer.get_feature_names()

# 데이터를 쉽게 보기 위해 DataFrame으로 변환
pd.DataFrame(X_train[0:10000].toarray(), columns=vocabulary).head()

,abandon,ability,ability to,able,able to,about,about all,about an,about bad,about be,...,yourself,youth,youthful,yu,zany,zeal,zero,zhang,zombie,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# 
X_test = vectorizer.transform(test["Phrase"])
X_test

<66292x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 417332 stored elements in Compressed Sparse Row format>

In [14]:
y_train = train["Sentiment"]

print(y_train.shape)
y_train.head()

(156060,)


PhraseId
1    1
2    2
3    2
4    2
5    2
Name: Sentiment, dtype: int64

#### SGDClassifier (Stochastic Gradient Descent, 경사하강법)

#### Parameters
* random_state : 랜덤 seed값을 지정. 지정하지 않을 경우 항상 다른 랜덤 값이 나온 (실험 시에는 값이 동일하도록 특정 숫자를 지정해준다)

In [15]:
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(random_state=37)  
model

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=37, shuffle=True,
       tol=None, verbose=0, warm_start=False)

## Scoring
모델의 성능을 측정

#### Cross Validation 
Training Set을 Sub Training Set과 Sub Test Set으로 나누어 검증한다. 나누는 방법은 여러 가지가 있는데 

In [16]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GroupKFold

kfold = GroupKFold(n_splits=5)

#y_predict = cross_val_predict(model, X_train, y_train, cv=5) 
y_predict = cross_val_predict(model, X_train, y_train, cv=kfold, groups=train["SentenceId"]) 

print(y_predict.shape)
y_predict

/Users/user/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/user/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/user/anacon

(156060,)


array([3, 2, 2, ..., 2, 2, 2])

In [17]:
from sklearn.metrics import accuracy_score

# 부분 집합 간의 정확도를 측정한다
score = accuracy_score(y_train, y_predict) 
print("Score = {0:0.6f}".format(score))

Score = 0.581103


In [16]:
#numpy : 선형 대수 연산을 위한 라이브러리. (숫자 연산 편하게)
import numpy as np

result = train.copy()
result["Sentiment(predict)"] = y_predict 

#예측값과 실제값 간의 차이를 구함 
result["Distance"] = result["Sentiment"] - result["Sentiment(predict)"]
result["Distance"] = np.abs(result["Distance"])

#차이가 크게 나는대로 정렬
result = result.sort_values(by="Distance", ascending=False)
result.head()

,SentenceId,Phrase,Sentiment,Phrase(Origin),Sentiment(predict),Distance
PhraseId,,,,,,
119884,6412,"arriv for an incongruou summer playoff , demon...",0,"arrived for an incongruous summer playoff , de...",4,4
28728,1331,' so sloppily written and cast that you can no...,0,'s so sloppily written and cast that you can n...,4,4
122159,6547,Hollywood End i the most disappoint Woody Alle...,0,Hollywood Ending is the most disappointing Woo...,4,4
29882,1385,you like an extreme action pack film with a hi...,4,you like an extreme action-packed film with a ...,0,4
110292,5841,That such a horrible movie could have sprung f...,0,That such a horrible movie could have sprung f...,4,4


In [17]:
# 결과를 저장
result[0:10000].to_csv("./output/result.csv")

In [18]:
# 모델에서 사용된 단어 저장
pd.DataFrame(vocabulary).to_csv("./output/vocabulary.csv")

## Predict
완성된 모델에 Test Feature를 입력하여 Test Label을 예측한다 

In [19]:
model.fit(X_train, y_train)

predictions = model.predict(X_test)

print(predictions.shape)
predictions

/Users/user/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


(66292,)


array([3, 3, 2, ..., 2, 2, 1])

## Submit (Kaggle 제출용) 

In [20]:
#kaggle 제출용 Sample파일을 읽어 그 형식에 맞게 답을 입력하여 제출
submissision = pd.read_csv("./input/sampleSubmission.csv", index_col="PhraseId")

print(submissision.shape)
submissision.head()

(66292, 1)


,Sentiment
PhraseId,
156061,2
156062,2
156063,2
156064,2
156065,2


In [21]:
submissision["Sentiment"] = predictions

print(submissision.shape)
submissision.head()

(66292, 1)


,Sentiment
PhraseId,
156061,3
156062,3
156063,2
156064,3
156065,2


In [22]:
submissision.to_csv("./output/baseline-submit.csv")